In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import logging
import yaml
import time
from email.policy import default as default_policy

In [ ]:
# # account credentials
# username = "youremailaddress@provider.com"
# password = "yourpassword"

# # imap_server = "outlook.office365.com"
# imap_server = "imap.gmail.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

### Connect/Login to IMAP server for Gmail

In [ ]:
def load_credentials(filepath):
    try:
        with open(filepath, 'r') as file:
            credentials = yaml.safe_load(file)
            user = credentials['user']
            password = credentials['password']
            return user, password
    except Exception as e:
        logging.error("Failed to load credentials: {}".format(e))
        raise

In [ ]:
def connect_to_gmail_imap(user, password):
    imap_url = 'imap.gmail.com'
    # imap_url = "outlook.office365.com"
    try:
        mail = imaplib.IMAP4_SSL(imap_url)
        mail.login(user, password)
        mail.select('inbox')  # Connect to the inbox.
        return mail
    except Exception as e:
        logging.error("Connection failed: {}".format(e))
        raise

In [ ]:
credentials = load_credentials('cred.yaml')
mail = connect_to_gmail_imap(*credentials)

In [ ]:
print(mail)

In [ ]:
def decode_str(string):
    decoded = email.header.decode_header(string)
    return email.header.make_header(decoded)
    #dh = decode_header(string)
    #default_charset = 'ASCII'
    #return ''.join([unicode(t[0], t[1] or default_charset) for t in dh ])

In [ ]:
result, data = mail.search(None, "ALL")
for num in data[0].split():
    result, msg_data_1 = mail.fetch(num, "(RFC822)")
    msg = email.message_from_bytes(msg_data_1[0][1], policy=default_policy)

    subject = msg['subject']
    result, msg_data = mail.fetch(num, "(INTERNALDATE)")
    assert result == "OK"
    assert len(msg_data) == 1, f"msg_data is not of length 1: {msg_data}"
    arrival_time = imaplib.Internaldate2tuple(msg_data[0])
    print(time.strftime("%a, %d %b %Y %H:%M:%S", arrival_time))

    from_email = msg['From']
    print("Subject:", subject, "from", from_email)
    print("Content-Type " , msg['Content-type'])

### Connect/Login to IMAP server for Outlook

In [ ]:
credentials = load_credentials('cred.yaml')
mail = connect_to_gmail_imap(*credentials)

In [ ]:
status, messages = mail.select("INBOX")
# number of top emails to fetch
N = 3
# total number of emails
messages = int(messages[0])
print(messages)

In [ ]:
for i in range(messages, messages-N, -1):
# for i in range(1, messages):
    # fetch the email message by ID
    res, msg = mail.fetch(str(i), "(RFC822)")
    result, time_data = mail.fetch(str(i), "(INTERNALDATE)")
    arrival_time = imaplib.Internaldate2tuple(time_data[0])
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # print(msg)
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            
            print("Subject:", subject)
            print("From:", From)
            print("Arrival Time: ",time.strftime("%a, %d %b %Y %H:%M:%S", arrival_time))

            
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            # if content_type == "text/html":
            #     # if it's HTML, create a new HTML file and open it in browser
            #     folder_name = clean(subject)
            #     if not os.path.isdir(folder_name):
            #         # make a folder for this email (named after the subject)
            #         os.mkdir(folder_name)
            #     filename = "index.html"
            #     filepath = os.path.join(folder_name, filename)
            #     # write the file
            #     open(filepath, "w").write(body)
            #     # open in the default browser
            #     webbrowser.open(filepath)
            print("="*100)
# close the connection and logout
mail.close()
mail.logout()

In [ ]:
Content-Type  multipart/alternative; boundary="_----------=_173298325022621221"
['Delivered-To', 'Received', 'X-Google-Smtp-Source', 'X-Received', 'ARC-Seal', 'ARC-Message-Signature', 'ARC-Authentication-Results', 'Return-Path', 'Received', 'Received-SPF', 'Authentication-Results', 'DKIM-Signature', 'DKIM-Signature', 'Date', 'To', 'From', 'Subject', 'Message-Id', 'MIME-Version', 'Content-Type', 'Content-Transfer-Encoding', 'List-Unsubscribe', 'Feedback-Id', 'X-Fncid', 'X-Injtime', 'X-Traffic-Type', 'X-Mailer', 'List-Unsubscribe-Post']
Sat, 30 Nov 2024 17:21:09
Subject: You’ve made the right choice, tankups93! from Groww <noreply@groww.in>
Content-Type  multipart/alternative; boundary="_----------=_173298366829200668"
['Delivered-To', 'Received', 'X-Google-Smtp-Source', 'X-Received', 'ARC-Seal', 'ARC-Message-Signature', 'ARC-Authentication-Results', 'Return-Path', 'Received', 'Received-SPF', 'Authentication-Results', 'DKIM-Signature', 'DKIM-Signature', 'Date', 'To', 'From', 'Subject', 'Message-Id', 'MIME-Version', 'Content-Type', 'Content-Transfer-Encoding', 'List-Unsubscribe', 'Feedback-Id', 'X-Fncid', 'X-Injtime', 'X-Traffic-Type', 'X-Mailer', 'List-Unsubscribe-Post']
Sun, 01 Dec 2024 03:13:46